In [ ]:
import pandas as pd
pd.set_option("display.max_columns",45)

**Read all input files using glob**

In [ ]:
import glob
#mylist = [f for f in glob.glob("*.txt")] ##this is list comprehension
txtfiles = []
for file in glob.glob("*.txt"):
    txtfiles.append(file)
##Remove everything after a _ in a string
    
#df['text_new'] = df['text'].str.split('::').str[0] dataframe solution
samples = [x.split('_')[0] for x in txtfiles] # this creates list of sample names 

**Check mutation in bedfile and write out  an excel output**

In [ ]:
for file in txtfiles:
    mut=pd.read_csv(file, sep="\t")
    mut["mutation_id"]= mut["GENENAME"]+ ":"+ mut["Index"]
    mut=mut.filter(regex="mutation_id|CLASS|AACHANGE|CALL")
    mut["SNV"]=mut.mutation_id.str.split(":").apply(lambda x :x[1])
    mut["Chromosome"]=mut.SNV.str.split(".").apply(lambda x :x[0])
    mut["position"]=mut.SNV.str.split(".").apply(lambda x :x[1])
    mut["Chromosome"]=mut.Chromosome.str.replace("chr","")
    mut['position'] = mut['position'].astype(float)
    cnv=pd.read_csv("bedfile.bed",sep="\t",header=None,
               names=["Chromosome","Start","End"])
    cnv["copy"]="yes"
    cnv_mut=mut.merge(cnv,on="Chromosome", how="outer")
    cnv_mut['position'] = cnv_mut['position'].astype(float)
    cnv_mut=cnv_mut[cnv_mut["position"].between(cnv_mut.pop("Start"),cnv_mut.pop("End"))]
    cnv_mut_select=cnv_mut[["Chromosome","position","copy"]]
    final=mut.merge(cnv_mut_select,how="left")
    final.to_excel(file.split('_')[0]+"_output_mutation_in_bedfile.xlsx", sheet_name='bedfile_mut', index=False)
    

**Mutation burden statistics**

In [ ]:
import glob
#mylist = [f for f in glob.glob("*.txt")] ##this is list comprehension
txtfiles = []
for file in glob.glob("*.xlsx"):
    txtfiles.append(file)
##Remove everything after a _ in a string
    
#df['text_new'] = df['text'].str.split('::').str[0] dataframe solution
samples = [x.split('_')[0] for x in txtfiles] # this creates list of sample names 

In [ ]:
Patient=[]
Total_mut=[]
Mut_in_bedfile=[]
Mut_not_in_bedfile=[]
Non_UTR_Total_mut=[]
Non_UTR_Total_mut_in_bedfile=[]
for file in txtfiles:
    df=pd.read_excel(file, index_col=None)
    t=file.split('_')[0]
    Patient.append(t)
    df1=df.filter(regex="copy|CLASS")
    l1=len(df1)
    Total_mut.append(l1)
    df2=df1[(df1["copy"] =="yes")]
    l2=len(df2)
    Mut_in_bedfile.append(l2)
    l3=l1-l2
    Mut_not_in_bedfile.append(l3)
    df3=df1[~df1['CLASS'].str.contains("UTR")]
    l4=len(df3)
    Non_UTR_Total_mut.append(l4)
    df4=df3[(df3["copy"] =="yes")]
    l5=len(df4)
    Non_UTR_Total_mut_in_bedfile.append(l5)

**Combine all list as list of lists**

In [ ]:
newlist=[Patient,Total_mut,Mut_in_bedfile,Mut_not_in_bedfile,Non_UTR_Total_mut,Non_UTR_Total_mut_in_bedfile]

In [ ]:
newlist

**Create index**

In [ ]:
index=["Patient","Total_mut","Mut_in_bedfile","Mut_not_in_bedfile","Non_UTR_Total_mut","Non_UTR_Total_mut_in_bedfile"]

**Convert list of list as dataframe with index**

In [ ]:
df = pd.DataFrame(newlist,index)

In [ ]:
df.head()

**Write out statistics in excel sheet with index and no header, since sample id is column name**

In [ ]:
writer = pd.ExcelWriter('Bedfile_mutation_statistics.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='val', index=True, columns=None)
writer.save()

**END**